In [1]:
import win32com.client as win32

In [2]:
path=r"C:\Users\jcleiva\Documents\Reportes/"
fname="MDR Industria Cad Sum - Interno.xlsx"

In [3]:
tiempo=(2023,8)

In [4]:
xl = win32.DispatchEx("Excel.Application")

wb = xl.workbooks.open(path+"/"+fname)
xl.Visible = True
#wb.RefreshAll()


In [5]:
wb.SlicerCaches("SegmentaciónDeDatos_Fecha").VisibleSlicerItemsList = ["[MDR].[Fecha].&[{}-{:02d}-01T00:00:00]".format(tiempo[0]-1,tiempo[1]),
                                                                       "[MDR].[Fecha].&[{}-{:02d}-01T00:00:00]".format(tiempo[0],tiempo[1])]

In [6]:
wb.SlicerCaches("SegmentaciónDeDatos_Fecha1").VisibleSlicerItemsList = ["[MDR].[Fecha].&[{}-{:02d}-01T00:00:00]".format(tiempo[0],tiempo[1])]

In [7]:
fechas=[]
val=False
for a in wb.SlicerCaches("SegmentaciónDeDatos_Fecha2").SlicerCacheLevels(1).SlicerItems:
    if a.Value =="1/{:02d}/{}".format(tiempo[1],tiempo[0]-1):
        val=True
    if val:
        fechas.append('[MDR].[Fecha].&[{}-{}-01T00:00:00]'.format(a.Value[-4:],a.Value.split("/")[1]))
    if a.Value =="1/{:02d}/{}".format(tiempo[1],tiempo[0]):
        val=False
wb.SlicerCaches("SegmentaciónDeDatos_Fecha2").VisibleSlicerItemsList = fechas

In [8]:
path=r"C:\Users\jcleiva\Documents\Reportes\Borradores/"
fname="P&G Cad Sum.pptx"

pwp = win32.Dispatch("PowerPoint.Application")

ppt = pwp.presentations.open(path+"/"+fname)

In [9]:
#,"V10","V26","V19","V23"
dSlides={"Cascada General":["Group 2","T6:X29",4],
        "58 - Comidas Preparadas":["Group 14","T6:X29",9],
        "63 - Bebidas":["Group 2","T6:X29",22],
        "68 - Charcuteria":["Group 2","T6:X29",12],
         "80 - Carnes Frias":["Group 2","T6:X29",25],
         "81 - Panaderia Ind":["Group 2","T6:X29",28],
         "86 - Carnes Rojas":["Group 2","T6:X29",15],
         "88 - Panaderia Fresca":["Group 2","T6:X29",18],
        }

In [10]:
for sd in dSlides.keys():
    wb.sheets(sd).Shapes(dSlides[sd][0]).Copy()
    a=ppt.Slides.Item(dSlides[sd][2]).Shapes.PasteSpecial(6)
    a.Left=15
    a.Top=130
    
    wb.sheets(sd).Range(dSlides[sd][1]).Copy()
    a=ppt.Slides.Item((dSlides[sd][2])).Shapes.PasteSpecial(DataType=1, Link=False)
    a.ScaleWidth(0.85,0)
    a.Left=605
    a.Top=130
    
    if sd != "Cascada General":

        vlr="{:,}".format(int(wb.sheets(sd).Range("V10").value)).replace(",",".")
        ppt.Slides.Item(dSlides[sd][2]+1).Shapes("IngNeto").TextFrame.TextRange.Text=vlr

        vlr="{:.1f}%".format(wb.sheets(sd).Range("V26").value*100)
        ppt.Slides.Item(dSlides[sd][2]+1).Shapes("varVN").TextFrame.TextRange.Text=vlr

        vlr="{:.1f}%".format(wb.sheets(sd).Range("V19").value*100)
        ppt.Slides.Item(dSlides[sd][2]+1).Shapes("MgnBto").TextFrame.TextRange.Text=vlr

        vlr="{:.1f}%".format(wb.sheets(sd).Range("V23").value*100)
        ppt.Slides.Item(dSlides[sd][2]+1).Shapes("MgnUCS").TextFrame.TextRange.Text=vlr
        
        wb.sheets(sd).Shapes("gMgnBto").Copy()
        a=ppt.Slides.Item(dSlides[sd][2]+1).Shapes.PasteSpecial(6)
        a.Left=220
        a.Top=273

        wb.sheets(sd).Shapes("gVarVN").Copy()
        a=ppt.Slides.Item(dSlides[sd][2]+1).Shapes.PasteSpecial(6)
        a.Left=220
        a.Top=405
        
        wb.sheets(sd).Range("BS4").CurrentRegion.Copy();
        a=ppt.Slides.Item(dSlides[sd][2]+1).Shapes.PasteSpecial(2)

        a.ScaleWidth(0.85,0)
        a.Top=60
        a.Left=185

        wb.sheets(sd).Range("BY4").CurrentRegion.Copy();
        a=ppt.Slides.Item(dSlides[sd][2]+1).Shapes.PasteSpecial(2)

        a.ScaleWidth(0.85,0)
        a.Top=60
        a.Left=570

In [11]:
meses={1:"Enero",
       2:"Febrero",
       3:"Marzo",
       4:"Abril",
       5:"Mayo",
       6:"Junio",
       7:"Julio",
       8:"Agosto",
       9:"Septiembre",
       10:"Octubre",
       11:"Noviembre",
       12:"Diciembre",
      }

In [12]:
ppt.Slides.Item(1).Shapes("CuadroTexto 10").TextFrame.TextRange.Text = "{} - {}".format(meses[tiempo[1]],tiempo[0])
ppt.Slides.Item(4).Shapes("CuadroTexto 14").TextFrame.TextRange.Text = "Resultado general {}-{}".format(meses[tiempo[1]][:3].lower(),str(tiempo[0])[-2:])
ppt.Slides.Item(5).Shapes("CuadroTexto 34").TextFrame.TextRange.Text = "Resultado general {}-{}".format(meses[tiempo[1]][:3].lower(),str(tiempo[0])[-2:])

In [13]:
ppt.SaveAs(r"C:\Users\jcleiva\Documents\Reportes\Presentaciones/{}{:02d}. MDR Cad Sum.pptx".format(tiempo[0],tiempo[1]))
ppt.close()

xl.CutCopyMode = False
wb.Close(True)